In [2]:
import cv2
import numpy as np
import pytesseract
import os
import re
import pandas as pd

pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import image
import matplotlib.pylab as plt
import tensorflow as tf

folder = r"../../../1.DATA/3.forms_images/Registry2008/"
year = '2008'
forms_list = os.listdir(folder)

# print(forms_list)

# Load the Model 

from tensorflow.keras.models import load_model
model = load_model('checkboxes_classifier.h5')

roi =[
    [(482,991),(495,1003), 'box', 'RemediationMeasures.Others'],
    [(251,991),(263,1003), 'box', 'RemediationMeasures.ContaminatedSoil/WasteCover'],
    [(47,991),(60,1003), 'box', 'RemediationMeasures.ReductiveDichlorination'],
    [(482,970),(495,983), 'box', 'RemediationMeasures.MonitoredNaturalAttenuation'],
    [(251,970),(263,983),'box', 'RemediationMeasures.GeotechnicalEncapsulation'],
    [(47,970),(60,983), 'box', 'RemediationMeasures.MultiphaseExtraction'],
    [(482,952),(495,964), 'box', 'RemediationMeasures.Biopile'],
    [(251,952),(263,964),'box', 'RemediationMeasures.FreePhaseRecovery'],
    [(47,952),(60,964),'box', 'RemediationMeasures.Bioventing'],
    [(251,935),(263,947),'box', 'RemediationMeasures.RemovalOfSoil/Residues'],
    [(483,932),(496,945),'box', 'RemediationMeasures.Phytoremediation'],
    [(47,932),(60,944),'box', 'RemediationMeasures.Biosparging'],
    [(482,915),(495,928),'box', 'RemediationMeasures.Bioremediation'],
    [(251,915),(263,928),'box', 'RemediationMeasures.SoilWashing'],
    [(47,915),(60,928), 'box', 'RemediationMeasures.AirSparging'],
    [(482,895),(495,908),'box', 'RemediationMeasures.Hydraulicbarrier'],
    [(251,895),(263,908), 'box', 'RemediationMeasures.ReactiveBarrier'],
    [(47,895),(60,908),'box', 'RemediationMeasures.SoilVapourExtraction'],
    [(482,878),(495,890),'box', 'RemediationMeasures.PhysicalBarrier'],
    [(251,878),(263,890),'box', 'RemediationMeasures.ChemicalOxidation/Reduction'],
    [(47,878),(60,890), 'box', 'RemediationMeasures.Pump&Treat'],
    
    [(690,843),(703,855),'box','InstitutionalControlMeasures.ConstructionWorkers.Implemented'],
    [(615,843),(627,855),'box', 'InstitutionalControlMeasures.ConstructionWorkers.CommunicatedToEA'],
    [(513,843),(526,855),'box', 'InstitutionalControlMeasures.ConstructionWorkers.RiskAssessment|InterventionPlan'],
    [(690,825),(703,837),'box', 'InstitutionalControlMeasures.BuildingsUse.Implemented'],
    [(613,825),(626,837),'box', 'InstitutionalControlMeasures.BuildingsUse.CommunicatedToEA'],
    [(513,825),(526,837),'box', 'InstitutionalControlMeasures.BuildingsUse.RiskAssessment|InterventionPlan'],
    [(690,807),(703,819), 'box', 'InstitutionalControlMeasures.FoodComsumption.Implemented'],
    [(615,809),(627,821),'box','InstitutionalControlMeasures.FoodComsumption.CommunicatedToEA'],
    [(513,809),(526,821),'box', 'InstitutionalControlMeasures.FoodComsumption.RiskAssessment|InterventionPlan'],
    [(690,789),(703,801),'box', 'InstitutionalControlMeasures.SurfaceWaterUse.Implemented'],
    [(615,790),(627,803),'box', 'InstitutionalControlMeasures.SurfaceWaterUse.CommunicatedToEA'],
    [(513,790),(526,803),'box', 'InstitutionalControlMeasures.SurfaceWaterUse.RiskAssessment|InterventionPlan'],
    [(690,771),(703,783),'box', 'InstitutionalControlMeasures.GroundwaterUse.Implemented'],
    [(615,771),(627,784),'box', 'InstitutionalControlMeasures.GroundwaterUse.CommunicatedToEA'],
    [(513,771),(526,784),'box', 'InstitutionalControlMeasures.GroundwaterUse.RiskAssessment|InterventionPlan'],
    [(690,750),(703,764), 'box', 'InstitutionalControlMeasures.SoilUse.Implemented'],
    [(615,752),(627,765), 'box', 'InstitutionalControlMeasures.SoilUse.CommunicatedToEA'],
    [(513,752),(526,765), 'box', 'InstitutionalControlMeasures.SoilUse.RiskAssessment|InterventionPlan'],
    
    [(60,842),(73,854), 'box', 'EmergencyMeasures.BanOnFoodConsumption'],
    [(60,824),(73,837),'box', 'EmergencyMeasures.ExcavationBan'],
    [(60,807),(73,820),'box', 'EmergencyMeasures.BuildingsBan'],
    [(60,792),(73,804),'box', 'EmergencyMeasures.Closing/InterdictionOfSupplyWells'],
    [(60,774),(73,787), 'box', 'EmergencyMeasures.RemovalOfmaterials'],
    [(60,757),(73,770),'box', 'EmergencyMeasures.EnvironmentalMonitoring'],
    [(60,742),(73,754),'box', 'EmergencyMeasures.MonitoringTheExplosivenessIndex'],
    [(60,724),(73,737),'box', 'EmergencyMeasures.Ventilation/ExhaustionOfConfinedSpaces'],
    [(60,707),(73,720), 'box', 'EmergencyMeasures.IsolationOfTheArea'],
    
# #     
    [(44,668),(56,680), 'box', 'Contaminants.POPs'],
    [(44,649),(56,662),'box', 'Contaminants.FreePhase'],
    [(645,632),(657,644), 'box', 'Contaminants.Others'],
    [(645,607),(657,620), 'box', 'Contaminants.Microbiological'],
    [(645,583),(657,596), 'box', 'Contaminants.Radionuclides'],
    [(645,561),(657,573),'box', 'Contaminants.Anilines'],
    [(645,536),(657,549),'box', 'Contaminants.Dioxins&Furans'],
    [(645,514),(657,526),'box', 'Contaminants.Phthalates'],
    [(645,489),(657,502), 'box', 'Contaminants.Biocides'],
    [(645,465),(657,478),'box', 'Contaminants.Fenols'],

    
    

    
    [(395,652),(407,665),'box', 'Contaminants.Methane/OtherGazes'],
    [(394,631),(406,644),'box', 'Contaminants.PCBs'],
    [(394,607),(406,620),'box', 'Contaminants.PAHs'],
    [(394,585),(406,597),'box', 'Contaminants.HalogenatedAromaticSolvents'],
    [(394,560),(406,573),'box', 'Contaminants.AromaticSolvents'],
    [(394,538),(406,550),'box', 'Contaminants.HalogenatedSolvents'],
    [(394,514),(406,526),'box', 'Contaminants.OtherInorganic'],
    [(394,489),(406,502),'box', 'Contaminants.Metals'],
    [(394,467),(406,479),'box', 'Contaminants.AutomotiveFuels'],
    
    [(289,616),(301,629),'box', 'ImpactedMedia.Biota.Outside'],
    [(207,616),(219,629),'box', 'ImpactedMedia.Biota.Inside'],
    [(289,599),(301,611),'box', 'ImpactedMedia.Air.Outside'],
    [(207,599),(219,611),'box', 'ImpactedMedia.Air.Inside'],
    [(287,582),(300,594),'box', 'ImpactedMedia.Sediments.Outside'],
    [(206,582),(218,594),'box', 'ImpactedMedia.Sediments.Inside'],
    [(287,562),(300,575),'box', 'ImpactedMedia.Groundwater.Outside'],
    [(206,562),(218,575),'box', 'ImpactedMedia.Groundwater.Inside'],
    [(287,546),(300,558),'box', 'ImpactedMedia.SurfaceWater.Outside'],
    [(206,546),(218,558),'box', 'ImpactedMedia.SurfaceWater.Inside'],
    [(287,529),(300,541),'box', 'ImpactedMedia.Soil.Outside'],
    [(206,529),(218,541),'box', 'ImpactedMedia.Soil.Inside'],
    [(287,509),(300,523),'box', 'ImpactedMedia.SuperficialSoil.Outside'],
    [(206,509),(218,523),'box', 'ImpactedMedia.SuperficialSoil.Inside'],
    
    
    [(287,424),(300,437),'box', 'ContaminationSource.Accidents'],
    [(39,421),(52,434),'box', 'ContaminationSource.Disposal'],
    [(416,421),(429,434),'box', 'ContaminationSource.Unknown'],
    [(175,421),(187,434),'box','ContaminationSource.Infiltration'],
    [(588,402),(600,415),'box', 'ContaminationSource.WastewaterTreatment'],
    [(415,402),(428,415),'box','ContaminationSource.AtmosphericEmissions'],
    [(289,402),(301,415),'box', 'ContaminationSource.Maintenance'],
    [(173,402),(186,415),'box', 'ContaminationSource.Production'],
    [(39,403),(52,415),'box', 'ContaminationSource.Storage'],
    
    [(390,355),(402,368),'box', 'ManagementPhase.ClosingMonitoring1'],
    [(390,337),(402,349),'box', 'ManagementPhase.RemediationWithEfficiency&EfficacyMonitoring1'],
    [(390,317),(402,329),'box', 'ManagementPhase.RemediationProject'],
    [(390,296),(402,309),'box', 'ManagementPhase.InterventionPlan'],
    [(390,278),(402,290),'box', 'ManagementPhase.RiskAssessment/RiskManagement'],
    [(390,260),(402,273),'box', 'ManagementPhase.DetailedInvestigation'],
    [(390,237),(402,250),'box','ManagementPhase.ConfirmatoryInvestigation1'],
    [(390,219),(402,231),'box', 'ManagementPhase.PreliminaryAssessment'],
    
    [(55,333),(68,345),'box', 'ManagementPhase.ClosingMonitoring'],
    [(55,308),(68,321),'box', 'ManagementPhase.RemediationWithEfficiency&EfficacyMonitoring'],
    [(55,290),(68,302),'box', 'ManagementPhase.DetailedInvestigation&InterventionPlan'],
    [(55,265),(68,278),'box', 'ManagementPhase.ConfirmatoryInvestigation'],
    [(55,241),(68,254),'box', 'ManagementPhase.LeakRemovalMeasures'],
    [(55,219),(68,231), 'box', 'ManagementPhase.OccurenceAssessment'],
    
    [(618,177),(631,190), 'box', 'Reuse'],
    [(667,142),(679,153),'box', 'Activity.Unknown'],
    [(588,139),(599,151),'box', 'Activity.Agriculture'],
    [(493,139),(505,151),'box', 'Activity.Accidents'],
    [(414,139),(426,151), 'box', 'Activity.Residue'],
    [(280,138),(291,151),'box', 'Activity.Gas_Station'], 
    [(202,139),(213,151),'box', 'Activity.Commerce'],
    [(115,139),(127,151),'box', 'Activity.Industry']]
data_boxes = []
pixels_check= []
value = 0

img = cv2.imread(folder+'/'+forms_list[0])
img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
img = cv2.resize(img,(0,0),fx=1,fy=1)
#forms_list = ['CETESB_registry_2019-1.jpg', 'CETESB_registry_2019-10.jpg',
#              'CETESB_registry_2019-100.jpg', 'CETESB_registry_2019-1000.jpg', 'CETESB_registry_2019-1001.jpg']
for i,r in enumerate(roi):
    x=r[0][0]
    y=r[0][1]
    cv2.rectangle(img,(r[0][0],r[0][1]),(r[1][0],r[1][1]),(0,255,0),2)
    cv2.putText(img=img, text=r[3],org =(x-2,y-2), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=0.25, color=(0, 0, 0),thickness=1)
    
cv2.imwrite(f'Detected_Textboxes_{year}_updated.jpg',img)
for j,y in enumerate(forms_list):
    data = []
    pixels = []
    img = cv2.imread(folder+"/"+ y)
    s_img = cv2.resize(img, (0, 0), fx=1,fy=1)
#     cv2.imwrite()
#     cv2.imshow(y,s_img)
#     cv2.waitKey(0)


    print(f'######### Extracting Data from {j} ##########')
    regex = re.compile(r'\d+')
    form_nb = regex.findall(y)[1]
    data.append(year)
    data.append(form_nb)
#     pixels.append(form_nb)
    for x,r in enumerate(roi):
        
        imgCrop = s_img[r[0][1]:r[1][1],r[0][0]:r[1][0]]
 
        gray = cv2.cvtColor(imgCrop,cv2.COLOR_BGR2GRAY)

#         n_white_pix = np.sum(gray == 255)

#         pixels.append(n_white_pix)
        imgCrop_resized = cv2.resize(imgCrop, (112,112))
        
        X = image.img_to_array(imgCrop_resized)
        X = np.expand_dims(X,axis=0)
        images = np.vstack([X])
        val = model.predict(images)
        if val == 0:
            ### box is checked ###
            value = 1
            data.append(value)
        else: 
            ### box is not checked ###
            value = 0
            data.append(value)                
                
                
#     print(data)


#     pixels_check.append(pixels)
    data_boxes.append(data)

list_columns = [r[3] for r in (roi)]
list_columns.insert(0,'FormNb')
list_columns.insert(0,'Year')
# # Create the pandas DataFrame
# columns = ['FormNb', 'Activity.Industry', 'Activity.Commerce', 'Activity.Gas_Station', 'Activity.Residue', 'Activity.Accidents', 
#            'Activity.Agriculture', 'Activity.Unknown', 'Reuse', 'ManagementPhase.OccurenceAssessment', 
#            'ManagementPhase.LeakRemovalMeasures', 'ManagementPhase.ConfirmatoryInvestigation', 
#            'ManagementPhase.DetailedInvestigation&InterventionPlan', 'ManagementPhase.RemediationWithEfficiency&EfficacyMonitoring', 
#            'ManagementPhase.ClosingMonitoring', 'ManagementPhase.PreliminaryAssessment', 'ManagementPhase.ConfirmatoryInvestigation1',
#            'ManagementPhase.DetailedInvestigation', 'ManagementPhase.RiskAssessment', 'ManagementPhase.InterventionPlan', 
#            'ManagementPhase.RemediationProject', 'ManagementPhase.RemediationWithEfficiency&EfficacyMonitoring1', 
#            'ManagementPhase.ClosingMonitoring1', 'ContaminationSource.Storage', 'ContaminationSource.Disposal', 
#            'ContaminationSource.Production', 'ContaminationSource.Infiltration', 'ContaminationSource.Maintenance', 
#            'ContaminationSource.Accidents', 'ContaminationSource.AtmosphericEmissions', 'ContaminationSource.Unknown', 
#            'ContaminationSource.WastewaterTreatment', 'ImpactedMedia.SuperficialSoil.Inside', 'ImpactedMedia.SuperficialSoil.Outside', 
#            'ImpactedMedia.Soil.Inside', 'ImpactedMedia.Soil.Outside', 'ImpactedMedia.SurfaceWater.Inside',
#            'ImpactedMedia.SurfaceWater.Outside', 'ImpactedMedia.Groundwater.Inside', 'ImpactedMedia.Groundwater.Outside', 
#            'ImpactedMedia.Sediments.Inside', 'ImpactedMedia.Sediments.Outside', 'ImpactedMedia.Air.Inside', 
#            'ImpactedMedia.Air.Outside', 'ImpactedMedia.Biota.Inside', 'ImpactedMedia.Biota.Outside', 'Contaminants.Metals',
#            'Contaminants.OtherInorganic', 'Contaminants.HalogenatedSolvents', 'Contaminants.AromaticSolvents', 
#            'Contaminants.HalogenatedAromaticSolvents', 'Contaminants.PAHs', 'Contaminants.PCBs', 'Contaminants.Methane',
#            'Contaminants.AutomotiveFuels', 'Contaminants.OtherVapours/Gazes', 'Contaminants.Fenols', 'Contaminants.Biocides',
#            'Contaminants.Phthalates', 'Contaminants.Dioxins&Furans', 'Contaminants.Anilines', 'Contaminants.Radionuclides', 
#            'Contaminants.Microbiological', 'Contaminants.TPH', 'Contaminants.Others', 'Contaminants.FreePhase', 'Contaminants.POPs',
#            'EmergencyMeasures.IsolationOfTheArea', 'EmergencyMeasures.Ventilation/ExhaustionOfConfinedSpaces',
#            'EmergencyMeasures.MonitoringTheExplosivenessIndex', 'EmergencyMeasures.EnvironmentalMonitoring', 
#            'EmergencyMeasures.RemovalOfmaterials', 'EmergencyMeasures.Closing/InterdictionOfSupplyWells', 
#            'EmergencyMeasures.BuildingsBan', 'EmergencyMeasures.ExcavationBan', 'EmergencyMeasures.BanOnFoodConsumption',
#            'InstitutionalControlMeasures.SoilUse.RiskAssessment|InterventionPlan', 
#            'InstitutionalControlMeasures.SoilUse.CommunicatedToEA', 'InstitutionalControlMeasures.SoilUse.Implemented',
#            'InstitutionalControlMeasures.GroundwaterUse.RiskAssessment|InterventionPlan', 
#            'InstitutionalControlMeasures.GroundwaterUse.CommunicatedToEA', 'InstitutionalControlMeasures.GroundwaterUse.Implemented', 
#            'InstitutionalControlMeasures.SurfaceWaterUse.RiskAssessment|InterventionPlan', 
#            'InstitutionalControlMeasures.SurfaceWaterUse.CommunicatedToEA', 'InstitutionalControlMeasures.SurfaceWaterUse.Implemented',
#            'InstitutionalControlMeasures.FoodConsumption.RiskAssessment|InterventionPlan', 
#            'InstitutionalControlMeasures.FoodConsumption.CommunicatedToEA', 'InstitutionalControlMeasures.FoodConsumption.Implemented', 
#            'InstitutionalControlMeasures.BuildingsUse.RiskAssessment|InterventionPlan', 
#            'InstitutionalControlMeasures.BuildingsUse.CommunicatedToEA', 'InstitutionalControlMeasures.BuildingsUse.Implemented',
#            'RemediationMeasures.Pump&Treat', 'RemediationMeasures.SoilVapourExtraction', 'RemediationMeasures.AirSparging', 
#            'RemediationMeasures.Biosparging', 'RemediationMeasures.Bioventing', 'RemediationMeasures.MultiphaseExtraction', 
#            'RemediationMeasures.ReductiveDichlorination', 'RemediationMeasures.InSituHeatTreatment', 'RemediationMeasures.ChemicalOxidation', 'RemediationMeasures.ChemicalReduction', 'RemediationMeasures.ReactiveBarrier', 'RemediationMeasures.SoilWashing', 'RemediationMeasures.RemovalOfSoil/Residues', 'RemediationMeasures.FreePhaseRecovery', 'RemediationMeasures.GeotechnicalEncapsulation', 'RemediationMeasures.ContaminatedSoil/WasteCover', 'RemediationMeasures.PhysicalBarrier', 'RemediationMeasures.Hydraulicbarrier', 'RemediationMeasures.Bioremediation', 'RemediationMeasures.Phytoremediation', 'RemediationMeasures.Biopile', 'RemediationMeasures.MonitoredNaturalAttenuation', 'RemediationMeasures.Others', 'RemediationMeasures.NoRemediationMeasures', 'EngineeringControlMeasures.ProjectSuitability', 'EngineeringControlMeasures.Waterproofing', 
#            'EngineeringControlMeasures.Paving', 'EngineeringControlMeasures.Others']
# pixel_boxes = pd.DataFrame(pixels_check, columns=list_columns)
data_boxes = pd.DataFrame(data_boxes, columns=list_columns)

# pixel_boxes.to_excel(f'../Final_Boxes/{year}.xlsx')
data_boxes.to_excel(f'../Final_Boxes/{year}_model_classifier.xlsx')


######### Extracting Data from 0 ##########
1/1 [==============================] - 14s 14s/step
######### Extracting Data from 1 ##########
1/1 [==============================] - 12s 12s/step


1/1 [==============================] - 12s 12s/step
######### Extracting Data from 2 ##########
1/1 [==============================] - 12s 12s/step


1/1 [==============================] - 12s 12s/step
######### Extracting Data from 3 ##########
1/1 [==============================] - 12s 12s/step
######### Extracting Data from 4 ##########
1/1 [==============================] - 12s 12s/step


1/1 [==============================] - 12s 12s/step
######### Extracting Data from 5 ##########
1/1 [==============================] - 12s 12s/step


1/1 [==============================] - 12s 12s/step
######### Extracting Data from 6 ##########
1/1 [==============================] - 13s 13s/step


...

1/1 [==============================] - 12s 12s/step
######### Extracting Data from 26 ##########
1/1 [==============================] - 13s 13s/step


1/1 [==============================] - 12s 12s/step
######### Extracting Data from 27 ##########
1/1 [==============================] - 12s 12s/step


1/1 [==============================] - 11s 11s/step
######### Extracting Data from 28 ##########
1/1 [==============================] - 12s 12s/step
######### Extracting Data from 29 ##########
1/1 [==============================] - 11s 11s/step


1/1 [==============================] - 15s 15s/step
######### Extracting Data from 30 ##########
1/1 [==============================] - 13s 13s/step


KeyboardInterrupt: 